In [13]:
!pip install pdfplumber
!pip install chromadb
!pip install sentence_transformers
!pip install langchain
!pip install --upgrade --quiet langchain-text-splitters tiktoken



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [62]:
import pdfplumber
import os
from langchain.text_splitter import TokenTextSplitter
from langchain.schema.document import Document

import chromadb
chroma_client = chromadb.Client()

In [2]:
pjoin = os.path.join
rag_data_folder = r"C:\Users\alkan\Downloads\RAG-Data"

In [4]:
rag_files = os.listdir(rag_data_folder)

rag_files[0]

'0008033v1.pdf'

In [25]:
pdf_path = pjoin(rag_data_folder,rag_files[0])

pages = []
with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        pages.append(text)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [30]:
def get_text_in_pages_from_pdf(pdf_path):
    pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            pages.append(text)
    return pages

In [23]:
len(text)

987

In [80]:
text_splitter = TokenTextSplitter(
chunk_size = 10, chunk_overlap = 2
)

texts = text_splitter.split_text(text)
texts[1]

'arenco, A. Ekert, K.'

In [28]:
collection = chroma_client.create_collection(name="Rag-DB")

In [ ]:
collection.add(
    documents=[],
    metadatas=[],
    ids=[]
)

In [31]:
def concat_pages(pages:list):
    return "".join(pages)

In [81]:
text_splitter = TokenTextSplitter(
chunk_size = 512, chunk_overlap = 150
)

In [84]:
pdf_path = pjoin(rag_data_folder,pdf_path)
pages = get_text_in_pages_from_pdf(pdf_path)
pdf_corpus = concat_pages(pages)
chunks = text_splitter.split_text(pdf_corpus)
chunks

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


['0002\nguA\n7\n1v3308000/hp-tnauq:viXra\nAddition on a Quantum Computer\nThomas G. Draper\nWritten: September 1, 1998 Revised: June 15, 2000\nAbstract\nA new method for computing sums on a quantum computer is intro-\nduced. This technique uses the quantum Fourier transform and reduces\nthe number of qubits necessary for addition by removing the need for\ntemporary carry bits. This approach also allows the addition of a clas-\nsical number to a quantum superposition without encoding the classical\nnumber in the quantum register. This method also allows for massive\nparallelization in its execution.\n1Contents\n1 Introduction 3\n2 Classical Addition 3\n3 The Quantum Fourier Transform 4\n4 The Approximate QFT 5\n5 Quantum Addition 6\n6 Implications 7\n21 Introduction\nTraditionally, addition algorithms designed for a quantum computer have mir-\nrored their classical counterparts[2, 6, 9], with the necessary extensions for re-\nversible computation. Faster quantum addition algorithms impl

In [ ]:
addable = {
    "documents" : [],
    "metadatas": [],
    "ids" : []       
                  }

page_num = 1
global_id = 0

for page in pages:
    
    chunks = text_splitter.split_text(page)
    addable["documents"].extend(chunks)
    addable["metadatas"].extend([{"source":"pdf_name_here","page":page_num} for i in range(len(chunks))])

    page_num+=1
    
    for i in range(len(chunks)):
        addable["ids"].append(str(global_id))
        global_id+=1

addable

{'documents': ['0002\nguA\n7\n1v3308000/hp-tnauq:viXra\nAddition on a Quantum Computer\nThomas G. Draper\nWritten: September 1, 1998 Revised: June 15, 2000\nAbstract\nA new method for computing sums on a quantum computer is intro-\nduced. This technique uses the quantum Fourier transform and reduces\nthe number of qubits necessary for addition by removing the need for\ntemporary carry bits. This approach also allows the addition of a clas-\nsical number to a quantum superposition without encoding the classical\nnumber in the quantum register. This method also allows for massive\nparallelization in its execution.\n1',
  'Contents\n1 Introduction 3\n2 Classical Addition 3\n3 The Quantum Fourier Transform 4\n4 The Approximate QFT 5\n5 Quantum Addition 6\n6 Implications 7\n2',
  '1 Introduction\nTraditionally, addition algorithms designed for a quantum computer have mir-\nrored their classical counterparts[2, 6, 9], with the necessary extensions for re-\nversible computation. Faster quantu

In [88]:
def create_addable(pages):

    addable = {
    "documents" : [],
    "metadatas": [],
    "ids" : []       
                  }

    global global_id
    
    page_num = 1

    for page in pages:
        
        chunks = text_splitter.split_text(page)
        addable["documents"].extend(chunks)
        addable["metadatas"].extend([{"source":"pdf_name_here","page":page_num} for i in range(len(chunks))])

        page_num+=1
        
        for i in range(len(chunks)):
            addable["ids"].append(str(global_id))
            global_id+=1

    return addable

In [87]:
global_id = 0

In [89]:
for pdf_name in rag_files:
    
    pdf_path = pjoin(rag_data_folder,pdf_path)
    pages = get_text_in_pages_from_pdf(pdf_path)
    addable = create_addable(pages)

    collection.add(
        documents=addable["documents"],
        metadatas=addable["metadatas"],
        ids=addable["ids"]
    )

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
C:\Users\alkan\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [05:10<00:00, 268kiB/s]   
CropBox missing from /Page, defaulting to MediaBox
CropBo

KeyboardInterrupt: 

In [57]:
chunks

[Document(metadata={}, page_content='0002\nguA\n7\n1v3308000/hp-tnauq:viXra\nAddition on a Quantum Computer\nThomas G. Draper\nWritten: September 1, 1998 Revised: June 15, 2000\nAbstract\nA new method for computing sums on a quantum computer is intro-\nduced. This technique uses the quantum Fourier transform and reduces\nthe number of qubits necessary for addition by removing the need for\ntemporary carry bits. This approach also allows the addition of a clas-\nsical number to a quantum superposition without encoding the classical\nnumber in the quantum register. This method also allows for massive\nparallelization in its execution.\n1Contents\n1 Introduction 3\n2 Classical Addition 3\n3 The Quantum Fourier Transform 4\n4 The Approximate QFT 5\n5 Quantum Addition 6\n6 Implications 7\n21 Introduction\nTraditionally, addition algorithms designed for a quantum computer have mir-\nrored their classical counterparts[2, 6, 9], with the necessary extensions for re-\nversible computation. Fast

In [ ]:
for pdf_name in rag_files:
    pdf_path = pjoin(rag_data_folder,pdf_path)
    pages = get_text_in_pages_from_pdf(pdf_path)
    pdf_corpus = concat_pages(pages)
    text_splitter.split_text(pdf_corpus)